<a href="https://colab.research.google.com/github/ReidelVichot/PUS2022_RVichot/blob/main/HW6/RandomForest_solution_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Following the paper "Examining the feasibility of using open data to benchmark building energy usage in cities: A data science and policy perspective" [Roth et al 2019](https://www.researchgate.net/publication/337074109_Examining_the_feasibility_of_using_open_data_to_benchmark_building_energy_usage_in_cities_A_data_science_and_policy_perspective), choose 2 cities in the dataset available through the paper's github repository https://github.com/Urban-Informatics-Lab/Open-Data-Benchmarking

This paper is an excellent example of reproducible research. 

I also created an example of using random forest classifier and regressors here https://github.com/fedhere/PUS2022_FBianco/blob/master/classdemo/COVID19LOS_featureExtractionRFexample.ipynb and there are links at the bottom of this notebook to useful functions and examples on the internet of applications of methods you will need to use.

1. Read the introduction to the paper and describe here (briefly) the goals of their analysis (and the analysis that you are about to reproduce). Max 200 words.

  Studies have shown gaps in buildings’ energy efficiency. Meaning that most efficient energetical technologies are not correctly implemented. This gap probably happens because of market barriers, imperfect information, hidden costs, regulatory failures, and behavioral negligence. 
  The urgency for action has motivated global initiatives like the United Nations Sustainable Development Goals (SDGs) and the Paris Climate Agreement that aim at reducing energy consumption. Some major cities worldwide committed to making all new buildings from 2030 carbon-neutral and older buildings carbon-neutral by 2050. 
  Reducing the energy consumption of buildings has important implications for environmental, economic, and social health. The increase in buildings’ energy efficiency can reduce the demand for electricity and, therefore, the amounts of carbon dioxide emitted to the atmosphere. Building energy benchmarking policies is among some of cities' responses to the benefits of implementing energy-efficient policies.  
  The paper (Roth et al., 2019) tries to answer the following questions: 
  *   Can city-wide energy benchmarking be conducted using only open data and how do such models compare to the current practice?
  *   What benchmarking data fields are important for cities to collect?
  *   What policies and programs improve the efficacy of energy benchmarking programs?


2. Choose 2 cities (any 2 cities except Seattle cause I am running some of the Seattle analysis below for guidance)

  *   Chicago
  *   DC
   

3. For each of the 2 cities reproduce their analysis by 

  3.1 gathering the original data from their repository (see below for Seattle example)

  3.2 clean the data according to their data preparation scheme, including one-hot-encoding categorical variables, except to impute missing data using KNearestNeighors instead of Gibbs sampling (see below)

  3.3 run a Random Forest (RF) Regressor to predict the total energy consumption.

  3.4 evaluate the RF model performance by printing the R2 score (the default score in SKLearn RF) for training and test set (discuss)

  3.5 plot the features sorted by their importance and identify the most important features. are they consistent between the 2 cities? are they consistent with the paper's result?
4 compare the result for the 2 cities (discuss)

5. Extra credit: 

  5.1 modify the target variable to a binary variable choosing the median of the variable values as the the energy threshold and predict whether the value is higher or lower than the threshold
  5.2 Run a Random Forest Classifier to predict if an observation is in the upper 50% percentile or lower 50% percentile (above or below the median threshold)
  5.3 Measure the performance of the model on training and test data
  5.4 Measure the feature importance and assess if the important feature have changed (discuss)

In [60]:
import pandas as pd
import numpy as np

# choose a city and read in the data

In [75]:
seattle = pd.read_csv("https://raw.githubusercontent.com/Urban-Informatics-Lab/Open-Data-Benchmarking/master/Seattle/seattle_final.csv")
dc = pd.read_csv("https://github.com/Urban-Informatics-Lab/Open-Data-Benchmarking/raw/master/DC/DC_final.csv")
chicago = pd.read_csv("https://github.com/Urban-Informatics-Lab/Open-Data-Benchmarking/raw/master/Chicago/chi_final.csv")

# inspect the dataframe

In [62]:
print("Seattle: there are (rows, columns) = ", (seattle.shape), "in the dataframe")
print("DC: there are (rows, columns) = ", (dc.shape), "in the dataframe")
print("Philadelphia: there are (rows, columns) = ", (chicago.shape), "in the dataframe")

Seattle: there are (rows, columns) =  (3376, 46) in the dataframe
DC: there are (rows, columns) =  (1455, 23) in the dataframe
Philadelphia: there are (rows, columns) =  (2688, 19) in the dataframe


In [63]:
seattle.describe()

,OSEBuildingID,DataYear,ZipCode,CouncilDistrictCode,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,...,SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),Comments,TotalGHGEmissions,GHGEmissionsIntensity
count,3376.000000,3376.0,3360.000000,3376.000000,3376.000000,3376.000000,3376.000000,3368.000000,3376.000000,3.376000e+03,...,3.371000e+03,3.370000e+03,3.367000e+03,3.367000e+03,3.367000e+03,3.367000e+03,3.367000e+03,0.0,3367.000000,3367.000000
mean,21208.991114,2016.0,98116.949107,4.439277,47.624033,-122.334795,1968.573164,1.106888,4.709123,9.483354e+04,...,5.403667e+06,5.276726e+06,2.745959e+05,1.086639e+06,3.707612e+06,1.368505e+04,1.368505e+06,NaN,119.723971,1.175916
std,12223.757015,0.0,18.615205,2.120625,0.047758,0.027203,33.088156,2.108402,5.494465,2.188376e+05,...,2.161063e+07,1.593879e+07,3.912173e+06,4.352478e+06,1.485066e+07,6.709781e+04,6.709781e+06,NaN,538.832227,1.821452
min,1.000000,2016.0,98006.000000,1.000000,47.499170,-122.414250,1900.000000,0.000000,0.000000,1.128500e+04,...,0.000000e+00,0.000000e+00,0.000000e+00,-3.382680e+04,-1.154170e+05,0.000000e+00,0.000000e+00,NaN,-0.800000,-0.020000
25%,19990.750000,2016.0,98105.000000,3.000000,47.599860,-122.350662,1948.000000,1.000000,2.000000,2.848700e+04,...,9.251286e+05,9.701822e+05,0.000000e+00,1.874229e+05,6.394870e+05,0.000000e+00,0.000000e+00,NaN,9.495000,0.210000
50%,23112.000000,2016.0,98115.000000,4.000000,47.618675,-122.332495,1975.000000,1.000000,4.000000,4.417500e+04,...,1.803753e+06,1.904452e+06,0.000000e+00,3.451299e+05,1.177583e+06,3.237538e+03,3.237540e+05,NaN,33.920000,0.610000
75%,25994.250000,2016.0,98122.000000,7.000000,47.657115,-122.319407,1997.000000,1.000000,5.000000,9.099200e+04,...,4.222455e+06,4.381429e+06,0.000000e+00,8.293178e+05,2.829632e+06,1.189033e+04,1.189034e+06,NaN,93.940000,1.370000
max,50226.000000,2016.0,98272.000000,7.000000,47.733870,-122.220966,2015.000000,111.000000,99.000000,9.320156e+06,...,8.739237e+08,4.716139e+08,1.349435e+08,1.925775e+08,6.570744e+08,2.979090e+06,2.979090e+08,NaN,16870.980000,34.090000


In [64]:
dc.describe()

,Unnamed: 0,ward,year_built,tax_record_floor_area,reported_gross_floor_area,water_use,electricity_use,natural_gas_use,energy_star_score,site_eui,weather_norm_site_eui,source_eui,weather_norm_source_eui,total_ghg_emissions,total_ghg_emissions_intensity,total_site_energy_KBTU,log_total_site_energy_KBTU
count,1455.000000,1455.000000,1455.000000,1.455000e+03,1.455000e+03,1.455000e+03,1.441000e+03,1.054000e+03,1184.000000,1455.000000,1393.000000,1455.000000,1393.000000,1454.000000,1454.000000,1.455000e+03,1455.000000
mean,793.997938,3.472852,1940.804811,2.046748e+05,1.897612e+05,5.109905e+04,2.760114e+06,5.853217e+04,62.090372,69.960206,70.745154,171.274433,171.302872,1318.734388,6.674278,1.344870e+07,15.905810
std,469.582655,2.047691,167.230361,2.451302e+05,2.436786e+05,6.163321e+05,4.599984e+06,2.617060e+05,26.792483,30.676317,31.476591,76.707287,76.884877,2458.471157,3.143349,2.843799e+07,0.991320
min,1.000000,1.000000,1000.000000,9.171000e+03,9.171000e+03,0.000000e+00,1.785000e+02,0.000000e+00,1.000000,1.100000,1.100000,1.200000,1.200000,3.100000,0.100000,5.729350e+04,10.955942
25%,380.500000,2.000000,1947.000000,7.164000e+04,7.111750e+04,1.958050e+03,7.435930e+05,1.121760e+04,43.750000,51.600000,51.900000,121.850000,121.600000,400.450000,4.800000,4.513227e+06,15.322523
50%,794.000000,2.000000,1971.000000,1.287720e+05,1.215530e+05,4.063800e+03,1.612028e+06,2.770698e+04,70.000000,65.900000,66.600000,163.300000,163.400000,757.100000,6.300000,8.156027e+06,15.914268
75%,1176.500000,5.000000,1994.000000,2.585310e+05,2.372130e+05,7.008700e+03,3.386553e+06,5.154631e+04,84.000000,82.850000,84.500000,206.450000,207.400000,1534.450000,7.800000,1.565570e+07,16.566343
max,2181.000000,8.000000,2017.000000,3.997572e+06,5.634890e+06,1.929391e+07,1.207430e+08,7.269109e+06,100.000000,213.600000,215.500000,618.600000,620.600000,64992.500000,30.600000,7.832497e+08,20.478962


In [65]:
chicago.describe()

,Unnamed: 0,ID,Gross.Floor.Area...Buildings..sq.ft.,Year.Built,X..of.Buildings,ENERGY.STAR.Score,Electricity.Use..kBtu.,Natural.Gas.Use..kBtu.,Site.EUI..kBtu.sq.ft.,Source.EUI..kBtu.sq.ft.,Weather.Normalized.Site.EUI..kBtu.sq.ft.,Weather.Normalized.Source.EUI..kBtu.sq.ft.,Total.GHG.Emissions..Metric.Tons.CO2e.,GHG.Intensity..kg.CO2e.sq.ft.,total_site_energy_KBTU,log_total_site_energy_KBTU
count,2688.000000,2688.000000,2.688000e+03,2688.000000,2688.000000,2688.000000,2.688000e+03,2.688000e+03,2688.000000,2688.000000,2658.000000,2658.000000,2673.000000,2673.000000,2.688000e+03,2688.000000
mean,1344.500000,184502.570312,2.448678e+05,1962.942336,1.405878,57.507812,1.076602e+07,1.172915e+07,94.208408,180.017820,99.133296,184.965500,2630.786495,10.103629,2.298315e+07,16.304023
std,776.103086,59873.199256,3.964045e+05,36.294735,5.201993,29.352026,2.495836e+07,2.722341e+07,125.939184,166.056539,127.267249,166.420203,5794.436792,9.002384,4.971672e+07,1.015335
min,1.000000,100001.000000,5.000000e+04,1872.000000,1.000000,1.000000,5.028900e+03,1.284000e+03,0.200000,0.200000,0.300000,0.300000,32.500000,0.400000,1.580000e+04,9.667765
25%,672.750000,120464.000000,7.500000e+04,1928.000000,1.000000,34.000000,1.947257e+06,3.182139e+06,61.800000,113.875000,65.400000,118.625000,602.100000,6.300000,5.838643e+06,15.580009
50%,1344.500000,173712.500000,1.237265e+05,1969.000000,1.000000,62.000000,4.024877e+06,5.561839e+06,79.400000,148.100000,84.400000,154.050000,1099.500000,8.300000,1.049675e+07,16.166576
75%,2016.250000,251356.500000,2.521352e+05,1997.000000,1.000000,83.000000,9.702889e+06,1.168477e+07,101.050000,196.575000,107.275000,201.575000,2435.700000,11.100000,2.202188e+07,16.907547
max,2688.000000,260184.000000,9.245333e+06,2016.000000,236.000000,100.000000,4.857050e+08,5.493302e+08,5637.700000,6001.500000,5637.700000,6001.500000,127610.000000,304.600000,1.155374e+09,20.867690


# TASK 1 Clean the data 
follow closely the second paragraph of Section 4 DATA. Name each step you do accordingly to the description in the paragraf
https://www.researchgate.net/publication/337074109_Examining_the_feasibility_of_using_open_data_to_benchmark_building_energy_usage_in_cities_A_data_science_and_policy_perspective 

you can also help yourself inspecting the final datasets posted on the github repo but name each action according to the paper narrative: for example, if you drop "Building ID" indicate that this is done as part of 
_We then cleaned the datasets by removing irrelevant building-identifying features (such as
address)_, or if dropping a variable with too many missing values _removing features that had missing values for greater than 40% of the buildings in the dataset._

In [ ]:
# Seattle
# summarize the number of rows with missing values for each column
for c in seattle.columns:  
  # count number of rows with missing values
  n_miss = seattle[c].isnull().sum()
  perc = n_miss / seattle.shape[0] * 100
  print('%s (%s):  Missing: %d (%.1f%%)' % (c, seattle[c].dtype, 
                                                  n_miss, perc))  
print("\n")  
# DC
for c in dc.columns:  
  n_miss = dc[c].isnull().sum()
  perc = n_miss / dc.shape[0] * 100
  print('%s (%s):  Missing: %d (%.1f%%)' % (c, dc[c].dtype, 
                                                  n_miss, perc))
print("\n")  
# chicago
for c in chicago.columns:  
  n_miss = chicago[c].isnull().sum()
  perc = n_miss / chicago.shape[0] * 100
  print('%s (%s):  Missing: %d (%.1f%%)' % (c, chicago[c].dtype, 
                                                  n_miss, perc))

In [68]:
for c in seattle.columns:
  # count number of rows with missing values
  n_miss = seattle[c].isnull().sum()
  perc = n_miss / seattle.shape[0] * 100
  if perc > 40:
    seattle.drop(c, axis=1, inplace=True)
print("there are (rows, columns) = ", (seattle.shape), "in the dataframe")

seattle = seattle[['log_total_site_energy_KBTU',"building_type",
                   "property_type_primary","zip","council_district_code",
                   "neighborhood","year_built","num_buildings","num_floors",
                   "gross_floor_area_total_SF","gross_floor_area_parking_SF",
                   "gross_floor_area_building_SF","property_type_first",
                   "gross_floor_area_property_type_first_SF"]]

there are (rows, columns) =  (3376, 39) in the dataframe


KeyError: ignored

In [ ]:
'''
import pandas as pd

dc = pd.read_csv("https://github.com/Urban-Informatics-Lab/Open-Data-Benchmarking/raw/master/DC/DC_final.csv")

columns = ["report_status","ward","postal_code","year_built","primary_ptype_self","primary_ptype_epa","tax_record_floor_area","reported_gross_floor_area","water_use","metered_areas_energy","metered_areas_water"]
dc[columns]
y_DC=DC['log_total_site_energy_KBTU']


chicago_features=c("num_buildings","year_built","property_type_primary_epa","floor_area_total_SF","h2o_use_allsources_KGAL","SALE_PR","SALE_TY","MV","TX_LND","TX_BLDG","XMPT_LND","XMPT_BLDG","CAT_CD","ZONE","FRT","DPT","TOT_AREA","TOP","GRG_TYP","GRG_SP","OFF_ST","VIEW_","STORIES","EXT_COND","NO_RM","NO_BD","NO_BATH","BASMT_SQFT","FIRE","INT_COND","TOT_LIV_AREA")
chicago_outputs=c("electricity_use_KBTU","energy_star_score","site_eui_KBTUSF","source_eui_KBTUSF","total_ghg_emissions_MTCO2e","total_site_energy_KBTU","log_total_site_energy_KBTU")
#split into features and outputs
'''

In [ ]:
for c in dc.columns:
  # count number of rows with missing values
  n_miss = dc[c].isnull().sum()
  perc = n_miss / dc.shape[0] * 100
  if perc > 40:
    dc.drop(c, axis=1, inplace=True)
# add anything else you want to do 
# our decisions and codes may differe here so if your results is not exactly like mine that is ok
# replace ... with your code
# dropping unnecessary identification columns
dc = dc[["report_status","ward","postal_code","year_built","primary_ptype_self",
          "primary_ptype_epa","tax_record_floor_area","reported_gross_floor_area",
          "water_use","metered_areas_energy","metered_areas_water", 'log_total_site_energy_KBTU']]
print("DC: there are (rows, columns) = ", (dc.shape), "in the dataframe")

for c in chicago.columns:
  n_miss = chicago[c].isnull().sum()
  perc = n_miss / chicago.shape[0] * 100
  if perc > 40:
    chicago.drop(c, axis=1, inplace=True)
chicago = chicago[["Primary.Property.Type","Gross.Floor.Area...Buildings..sq.ft.",
                   "Year.Built", "X..of.Buildings",'log_total_site_energy_KBTU']]
print("Chicago: there are (rows, columns) = ", (chicago.shape), "in the dataframe")


# TASK 2: transform input feature as needed
for example replace features with log features. Guide yourself with the text, the final dataset names, and also the code here https://github.com/Urban-Informatics-Lab/Open-Data-Benchmarking/blob/master/Lasso_RandomForest.Rmd

In [ ]:
for c in seattle.columns:
  
  # count number of rows with missing values
  n_miss = seattle[c].isnull().sum()
  perc = n_miss / seattle.shape[0] * 100
  print('%s (%s):  Missing: %d (%.1f%%)' % (c, seattle[c].dtype, 
                                                  n_miss, perc))
  
for c in dc.columns:
  
  # count number of rows with missing values
  n_miss = dc[c].isnull().sum()
  perc = n_miss / dc.shape[0] * 100
  print('%s (%s):  Missing: %d (%.1f%%)' % (c, dc[c].dtype, 
                                                  n_miss, perc))
  
for c in chicago.columns:
  
  # count number of rows with missing values
  n_miss = chicago[c].isnull().sum()
  perc = n_miss / chicago.shape[0] * 100
  print('%s (%s):  Missing: %d (%.1f%%)' % (c, chicago[c].dtype, 
                                                  n_miss, perc))
  

In [ ]:
print(seattle.describe()) #what shoudl you look at in the result below?
print(dc.describe())
print(chicago.describe())
#the overall description of the dataset

In [ ]:
seattle.info() #why is this helpful to print?
# data type of each feature

In [ ]:
dc.info() 

In [ ]:
chicago.info()

## Throw away data where the **target** variable is missing

I will remove each row where I do not have the target variable. To do that I use `df.dropna()` and the subset argument of `df.dropna()` set to the name of the variable. The `how` should be set to "any" cause you want to drop the row where any values in the subset is NaN (this is a subset of one column, so it is obvious, but imagine if you were passing more than one column to the call `dropna()`

In [ ]:
seattle["SiteEnergyUse(kBtu)"].isna().sum()

In [ ]:
dc["log_total_site_energy_KBTU"].isna().sum()

In [ ]:
chicago['log_total_site_energy_KBTU'].isna().sum()

In [ ]:
original_len = seattle.shape[0]
seattle.dropna(subset=['SiteEnergyUse(kBtu)'], inplace = True) #... # replace ... with your code

print("I lost {:.2f}% of seattle data".format((1 - (seattle.shape[0] / original_len)) * 100))

original_len = dc.shape[0]
dc.dropna(subset=['log_total_site_energy_KBTU'], inplace = True) #... # replace ... with your code

print("I lost {:.2f}% of dc data".format((1 - (dc.shape[0] / original_len)) * 100))

original_len = chicago.shape[0]
chicago.dropna(subset=['log_total_site_energy_KBTU'], inplace = True) #... # replace ... with your code

print("I lost {:.2f}% of chicago data".format((1 - (chicago.shape[0] / original_len)) * 100))


In [ ]:
# isolate the target variable first (endogenous)
y_seattle = seattle['SiteEnergyUse(kBtu)'].values
y_dc = dc['log_total_site_energy_KBTU'].values
y_chicago = chicago['log_total_site_energy_KBTU'].values

# and the input variables (exogenous)
X_seattle = seattle.drop('SiteEnergyUse(kBtu)', axis=1)
X_dc = dc.drop('log_total_site_energy_KBTU', axis=1)
X_chicago = chicago.drop('log_total_site_energy_KBTU', axis=1)



### separate categorical and numerical values
you will proceed to missing data imputation differently in the 2 cases

In [ ]:
# I am giving you the solution but please take note of how one does this!
X_seattle_numeric_data = X_seattle.select_dtypes(include=[np.number])
X_seattle_categorical_data = X_seattle.select_dtypes(exclude=[np.number])
X_dc_numeric_data = X_dc.select_dtypes(include=[np.number])
X_dc_categorical_data = X_dc.select_dtypes(exclude=[np.number])
X_chicago_numeric_data = X_chicago.select_dtypes(include=[np.number])
X_chicago_categorical_data = X_chicago.select_dtypes(exclude=[np.number])

In [ ]:
X_seattle_numeric_data.shape

In [ ]:
X_dc_numeric_data.shape

In [ ]:
X_chicago_numeric_data.shape

In [ ]:
X_seattle_numeric_data.head() ## is there anything suspicious? anything thas is in fact **not** a numerical variable? that will depend on how you did in TASK 1

In [ ]:
X_dc_numeric_data.head()

In [ ]:
X_chicago_numeric_data.head()

In [ ]:
X_seattle_categorical_data.shape

In [ ]:
X_dc_categorical_data.shape

In [ ]:
X_chicago_categorical_data.shape

## TASK 2.1: make a scatter plot to assess covariance of numerical variables

### IMPORTANT!! 

there are going to be some **very** collinear variable - zoom in and chek them out - what are they? You will need to remove variables that are _identical_, because you obviously do not want to imput the data twice in the model - it will not improve the model and it will impact the feature importance analysis.

What else looks like it should be removed? 

**Reason about it and explain your choices in the caption of the figure**



In [ ]:
# THIS TAKES A LOOOONG TIME! ~10 MINUTES ON COLAB FOR ME
import seaborn as sns
sns.pairplot(X_seattle_numeric_data)
# DELETE OUTPUT!

In [ ]:
X_seattle_numeric_data.head()

In [ ]:
# which rows should you drop?
X_seattle_numeric_data.drop(["Latitude",	"Longitude"], axis = 1)


In [ ]:
X_seattle_numeric_data.describe()

In [ ]:
# this does not have to be identical to my output as you may have dropped different variables
print("there are (rows, columns) = ", X_seattle_numeric_data.shape, "in the numerical variables")

As they do in the original research https://github.com/Urban-Informatics-Lab/Open-Data-Benchmarking/blob/master/Lasso_RandomForest.Rmd we should transform some variables into their log values 

their original code (which is in R not in python!) is
```
x_seattle$log_total_area=log(x_seattle$gross_floor_area_total_SF)
x_seattle$log_parking_area=log(x_seattle$gross_floor_area_parking_SF)
x_seattle$log_bldg_area=log(x_seattle$gross_floor_area_building_SF)
x_seattle$log_area_1=log(x_seattle$gross_floor_area_property_type_first_SF)

```
the names are a bit different


In [78]:
# note: recheck missing values 
# this is important: after I apply the log function to some variables cause log(0) = -infinity
print("there are {} missing or infinity values in the numerical data".format(X_seattle_numeric_data.isnull().sum().sum()))

there are 912 missing or infinity values in the numerical data


In [ ]:
# translated to python
X_seattle_numeric_data["PropertyGFATotal"] = np.log10(X_seattle_numeric_data["PropertyGFATotal"])
... #replace ... with your code

In [ ]:
print("number of infinite values:", (np.isinf(X_numeric_data)).sum().sum())

check what happened to missing values: they may have grown! because  log(0) = -infinity - fix infinities replacing them with NaN 

(in in https://github.com/Urban-Informatics-Lab/Open-Data-Benchmarking/blob/master/Lasso_RandomForest.Rmd they had replaced them with 0 but I object to that)

In [ ]:
# this has to return 0 now
X_numeric_data.replace(-np.inf, np.nan, inplace=True)
print("number of infinite values:", (np.isinf(X_numeric_data)).sum().sum())

In [ ]:
print("the numerical data contains (rows, columns) = ", X_numeric_data.shape)
print("there are {} missing values in the numerical data".format(X_numeric_data.isnull().sum().sum()))


you can use `KNNImputer` on the numerical variables, but KNNImputer would fail on categorical variables so you are left with NaNs on categorical variables, which you then need to drop before you run the Random Forest


You can impute separately the numerical and categorical variables, the numerical ones with KNNImputer and the categorical ones with SimpleImputer

## TASK 2.2 use Nearest Neighbor Imputer to impute missing values in numerical features

here we are deviating from the paper. NN imputation is simpler and more common, I think it may be more useful to you in general

In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(... #replace ... with your code
Xn = ...
Xn

## TASK 2.3 Impute missing categorial variables 
you will have to remove Nan values and replace them. 

You can use `sklearn.impute.SimpleImputer using strategy="most_frequent"` on all variables, but you will have a naif solution for numerical imputations

I prefer to replace them with "None" or a similar string, for which you can use the fillna method: ```df.fillna("None")``` and leave it at that or with a little more effort you can input values based on nearest neighbors or some other inputation scheme. any choice goes so long as it is well described and justified with words.

In [ ]:
### here you replace NaN in categorical with one of the options above
# your code here
... #replace ... with your code

# TASK 3 One Hot Encode the Categorical Variables

Hint: Once you have done the fit and the transformation, which gives you the transformed data according to the new encoding, what you get is normally a sparse matrix. In principle there is an argument `sparse` to `OneHotEncoder` which should prevent this and give you a dense matrix, which is much easier to handle (consider asking me or looking up what is a sparse matrix!). That attribute did not work for me so once I applied the new encoding I converted the result to a dense matrix as 
`Xc = Xc.todense()`

Finally, you will need to use a specific method to get the new names of the variables. Its a method of your model (which I called ohe below) and you can find it in this page here https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [ ]:
X_categorical_data.describe()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# DELETE
#your code here

ohe = OneHotEncoder(... #replace ... with your code
...
Xc.shape

In [ ]:
# there is a speific method of ohe that allows you to get the names of the features after the encoding is done
feature_name_cat = ohe...... #replace ... with your code

# Task 4 recombine your numerical and categorical imput features and split your data into training and testing

In [ ]:
# this dont need to be the same for you
Xn.shape

In [ ]:
# this dont need to be the same for you
Xc.shape

In [ ]:
X = np.hstack([Xn, Xc])
X.shape

In [ ]:
print("The dataset has {} features (!!!)".format(X.shape[1]))

In [ ]:
# make a train and test dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = ..... #replace ... with your code

# TASK 5 run a random forest REGRESSION model

report the score on the tranining and test data and identify the most important features

(hint: if you see "ValueError: Input contains NaN, infinity or a value too large for dtype('float64')." check 
1. np.isnan(Xn).sum().sum() and np.isnan(Xc).sum().sum() both return 0
2. np.isinf(Xn).sum().sum() returns 0
3. all the feature that had very large numbers (> 1e5) have been converted to the log of their values

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(... #replace ... with your code

In [ ]:
print ("the model test accuracy is {:.2}".format(rf.score(X_test, y_test)))

In [ ]:
#this is how you see the importance of the features
rf.feature_importances_[:10]

In [ ]:

feature_names = .... #replace ... with your code
len(feature_names)

In [ ]:
# plot the top ~50 features
import pylab as plt

sorted_idx = ... #replace ... with your code 

y_ticks = np.arange(0, len(feature_names))
fig, ax = plt.subplots(figsize=(10,20))
ax.barh(y_ticks, tree_feature_importances[sorted_idx])
ax.set_yticklabels(feature_names[sorted_idx])
ax.set_yticks(y_ticks)
ax.set_title("Random Forest Feature Importances (MDI)")
fig.tight_layout()
ax.set_ylim(550, len(feature_names))
plt.show()

#your plot may be different from mine

In [ ]:
# CAPTION HERE! How is your model, How is your feature importance results? what can you notice? is it interesting? is it insightful??

# TASK 6 do it for another city! note: the variable names wont be the same


# useful links and methods and functions I used: 

```df.dropna()``` to remove observation with missing variables

```df.drop(columns, axis=1)``` to remove features (columns)


```np.log10()``` and ```np.isinf()``` to convert to log space and find infinite values  

`np.hstack([X1, X2])` to concatenate two arrays


one hot encoder https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html and https://stackabuse.com/one-hot-encoding-in-python-with-pandas-and-scikit-learn/


KNN imputer https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html , https://www.analyticsvidhya.com/blog/2020/07/knnimputer-a-robust-way-to-impute-missing-values-using-scikit-learn/, and 
https://machinelearningmastery.com/knn-imputation-for-missing-values-in-machine-learning/  